In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Transformer(nn.Module):
  def __init__(self, src_embed, target_embed, encoder, decoder, generator):
    super(Transformer, self).__init__()
    self.src_embed = src_embed
    self.target_embed = target_embed
    self.encoder = encoder
    self.decoder = decoder
    self.generator = generator

  def encoder(self, src, src_mask):
    out = self.encoder(self.src_embed(src), src_mask)
    return out

  def decoder(self, target, encoder_out, target_mask, src_target_mask):
    out = self.decoder(self.target_embed(target), encoder_out, target_mask, src_target_mask)
    return out

  def forward(self, src, target):
    src_mask = self.make_src_mask(src)
    target_mask = self.make_target_mask(target)
    src_target_mask = self.make_src_target_mask(src, target)
    encoder_out = self.encoder(src, src_mask)
    decoder_out = self.decoder(target, encoder_out, target_mask, src_target_mask)
    out = self.generator(decoder_out)
    out = F.log_softmax(out, dim=-1)
    return out, decoder_out

  def make_pad_mask(self, query, key, pad_idx=1):
    # query: (n_batch, query_seq_len)
    # key: (n_batch, key_seq_len)
    query_seq_len, key_seq_len = query.size(1), key.size(1)

    key_mask = key.ne(pad_idx).unsqueeze(1).unsqueeze(2)  # (n_batch, 1, 1, key_seq_len)
    key_mask = key_mask.repeat(1, 1, query_seq_len, 1)    # (n_batch, 1, query_seq_len, key_seq_len)

    query_mask = query.ne(pad_idx).unsqueeze(1).unsqueeze(3)  # (n_batch, 1, query_seq_len, 1)
    query_mask = query_mask.repeat(1, 1, 1, key_seq_len)  # (n_batch, 1, query_seq_len, key_seq_len)

    mask = key_mask & query_mask
    mask.requires_grad = False
    return mask

  def make_src_mask(self, src):
    pad_mask = self.make_pad_mask(src, src)
    return pad_mask

  def make_subsequent_mask(query, key):
    # query: (n_batch, query_seq_len)
    # key: (n_batch, key_seq_len)
    query_seq_len, key_seq_len = query.size(1), key.size(1)

    tril = np.tril(np.ones((query_seq_len, key_seq_len), k=0)).astype('uint8')
    mask = torch.tensor(tril, dtype=torch.bool, requires_grad=False, device=query.device)
    return mask

  def make_target_mask(self, target):
    pad_mask = self.make_pad_mask(target, target)
    seq_mask = self.make_subsequent_mask(target, target)
    mask = pad_mask & seq_mask
    return mask

  def make_src_target_mask(self, src, target):
    pad_mask = self.make_pad_mask(target, src)
    return pad_mask
  

In [8]:
import copy

class Encoder(nn.Module):
  def __init__(self, encoder_block, n_layer):
    super(Encoder, self).__init__()
    self.layers = []
    for i in range(n_layer):
      self.layers.append(copy.deepcopy(encoder_block))

  def forward(self, src, src_mask):
    out = src
    for layer in self.layers:
      out = layer(out, src_mask)
    return out

In [37]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention, position_ff, norm, dr_rate=0):
    super(EncoderBlock, self).__init__()
    self.self_attention = self_attention 
    self.position_ff = position_ff
    self.residuals = [ResidualConnectionLayer(copy.deepcopy(norm), dr_rate) for _ in range(2)]

  def forward(self, src, src_mask):
    out = src
    out = self.residuals[0](out, lambda out: self.self_attention(query=out, key=out, value=out, mask=src_mask))
    out = self.residuals[1](out, self.position_ff)
    return out

In [32]:
import math

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, d_model, h, qkv_fc, out_fc, dr_rate=0):
    super(MultiHeadAttentionLayer, self).__init__()
    self.d_model = d_model
    self.h = h
    self.q_fc = copy.deepcopy(qkv_fc) # (d_embed, d_model)
    self.k_fc = copy.deepcopy(qkv_fc) # (d_embed, d_model)
    self.v_fc = copy.deepcopy(qkv_fc) # (d_embed, d_model)
    self.out_fc = out_fc              # (d_model, d_embed)
    self.dropout = nn.Dropout(p=dr_rate)

  def calculate_attention(self, query, key, value, mask):
    # query, key, value: (n_batch, seq_len, d_k)
    # mask: (n_batch, seq_len, seq_len)
    d_k = key.shape[-1]
    attention_score = torch.matmul(query, key.transpose(-2, -1))  # Q x K^T, (n_batch, seq_len, seq_len)
    attention_score = attention_score / math.sqrt(d_k)
    if mask is not None:
      attention_score = attention_score.masked_fill(mask==0, -1e9)
    attention_prob = F.softmax(attention_score, dim=-1) # (n_batch, h, seq_len, seq_len)
    attention_prob = self.dropout(attention_prob)
    out = torch.matmul(attention_prob, value) # (n_batch, h, seq_len, d_k)
    return out

  def forward(self, *args, query, key, value, mask=None):
    # query, key, value: (n_batch, seq_len, d_embed)
    # mask: (n_batch, seq_len, seq_len)
    # return value: (n_batch, h, seq_len, d_k)
    n_batch = query.size(0)

    def transform(x, fc):  # (n_batch, seq_len, d_embed)
      out = fc(x)          # (n_batch, seq_len, d_model)
      out = out.view(n_batch, -1, self.h, self.d_model//self.h) # (n_batch, seq_len, h, d_k)
      out = out.transpose(1, 2) # (n_batch, h, seq_len, d_k)
      return out

    query = transform(query, self.q_fc) # (n_batch, h, seq_len, d_k)
    key = transform(key, self.k_fc)     # (n_batch, h, seq_len, d_k)
    value = transform(value, self.v_fc) # (n_batch, h, seq_len, d_k)

    out = self.calculate_attention(query, key, value, mask) # (n_batch, h, seq_len, d_k)
    out = out.transpose(1, 2) # (n_batch, seq_len, h, d_k)
    out = out.contiguous().view(n_batch, -1, self.d_model) # (n_batch, seq_len, d_model)
    out = self.out_fc(out) # (n_batch, seq_len, d_embed)
    return out


In [35]:
class PositionWiseFeedForwardLayer(nn.Module):
  def __init__(self, fc1, fc2, dr_rate=0):
      super(PositionWiseFeedForwardLayer, self).__init__()
      self.fc1 = fc1   # (d_embed, d_ff)
      self.relu = nn.ReLU()
      self.dropout = nn.Dropout(p=dr_rate)
      self.fc2 = fc2 # (d_ff, d_embed)


  def forward(self, x):
      out = x
      out = self.fc1(out)
      out = self.relu(out)
      out = self.dropout(out)
      out = self.fc2(out)
      return out


In [34]:
class ResidualConnectionLayer(nn.Module):
  def __init__(self, norm, dr_rate=0):
      super(ResidualConnectionLayer, self).__init__()
      self.norm = norm
      self.dropout = nn.Dropout(p=dr_rate)

  def forward(self, x, sub_layer):
      out = x
      out = self.norm(out)
      out = sub_layer(out)
      out = self.dropout(out)
      out = out + x
      return out

In [39]:
class Decoder(nn.Module):
  def __init__(self, decoder_block, n_layer, norm):
    super(Decoder, self).__init__()
    self.n_layer = n_layer
    self.layers = nn.ModuleList([copy.deepcopy(decoder_block) for _ in range(self.n_layer)])
    self.norm = norm

  def forward(self, target, encoder_out, target_mask, src_target_mask):
      out = target
      for layer in self.layers:
          out = layer(out, encoder_out, target_mask, src_target_mask)
      out = self.norm(out)
      return out



In [38]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention, cross_attention, position_ff, norm, dr_rate=0):
      super(DecoderBlock, self).__init__()
      self.self_attention = self_attention
      self.cross_attention = cross_attention
      self.position_ff = position_ff
      self.residuals = [ResidualConnectionLayer(copy.deepcopy(norm), dr_rate) for _ in range(3)]

  def forward(self, target, encoder_out, target_mask, src_target_mask):
      out = target
      out = self.residuals[0](out, lambda out: self.self_attention(query=out, key=out, value=out, mask=target_mask))
      out = self.residuals[1](out, lambda out: self.cross_attention(query=out, key=encoder_out, value=encoder_out, mask=src_target_mask))
      out = self.residuals[2](out, self.position_ff)
      return out

In [36]:
class TransformerEmbedding(nn.Module):
  def __init__(self, token_embed, pos_embed, dr_rate=0):
      super(TransformerEmbedding, self).__init__()
      self.embedding = nn.Sequential(token_embed, pos_embed)
      self.dropout = nn.Dropout(p=dr_rate)

  def forward(self, x):
      out = x
      out = self.embedding(out)
      out = self.dropout(out)
      return out

In [26]:
class TokenEmbedding(nn.Module):
  def __init__(self, d_embed, vocab_size):
      super(TokenEmbedding, self).__init__()
      self.embedding = nn.Embedding(vocab_size, d_embed)
      self.d_embed = d_embed

  def forward(self, x):
      out = self.embedding(x) * math.sqrt(self.d_embed)
      return out

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_embed, max_len=256, device=torch.device("cpu")):
      super(PositionalEncoding, self).__init__()
      encoding = torch.zeros(max_len, d_embed)
      encoding.requires_grad = False
      position = torch.arange(0, max_len).float().unsqueeze(1)
      div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed))
      encoding[:, 0::2] = torch.sin(position * div_term)
      encoding[:, 1::2] = torch.cos(position * div_term)
      self.encoding = encoding.unsqueeze(0).to(device)

  def forward(self, x):
      _, seq_len, _ = x.size()
      pos_embed = self.encoding[:, :seq_len, :]
      out = x + pos_embed
      return out

In [29]:
def build_model(src_vocab_size, 
                target_vocab_size, 
                device=torch.device("cpu"), 
                max_len=256, 
                d_embed=512, 
                n_layer=6, 
                d_model=512, 
                h=8, 
                d_ff=2048):
    copy = copy.deepcopy

    src_token_embed = TokenEmbedding(
                                      d_embed = d_embed,
                                      vocab_size = src_vocab_size)
    target_token_embed = TokenEmbedding(
                                      d_embed = d_embed,
                                      vocab_size = target_vocab_size)
    pos_embed = PositionalEncoding(
                                    d_embed = d_embed,
                                    max_len = max_len,
                                    device = device)

    src_embed = TransformerEmbedding(
                                      token_embed = src_token_embed,
                                      pos_embed = copy(pos_embed))
    target_embed = TransformerEmbedding(
                                      token_embed = target_token_embed,
                                      pos_embed = copy(pos_embed))

    attention = MultiHeadAttentionLayer(
                                        d_model = d_model,
                                        h = h,
                                        qkv_fc = nn.Linear(d_embed, d_model),
                                        out_fc = nn.Linear(d_model, d_embed))
    position_ff = PositionWiseFeedForwardLayer(
                                                fc1 = nn.Linear(d_embed, d_ff),
                                                fc2 = nn.Linear(d_ff, d_embed))

    encoder_block = EncoderBlock(
                                  self_attention = copy(attention),
                                  position_ff = copy(position_ff))
    decoder_block = DecoderBlock(
                                  self_attention = copy(attention),
                                  cross_attention = copy(attention),
                                  position_ff = copy(position_ff))

    encoder = Encoder(
                      encoder_block = encoder_block,
                      n_layer = n_layer)
    decoder = Decoder(
                      decoder_block = decoder_block,
                      n_layer = n_layer)
    generator = nn.Linear(d_model, target_vocab_size)

    model = Transformer(
                        src_embed = src_embed,
                        target_embed = target_embed,
                        encoder = encoder,
                        decoder = decoder,
                        generator = generator).to(device)
    model.device = device

    return model